# Stock price forecasting


## Problem description

In this challenge, we desire to predict the short-term evolution of the stock market prices of five major companies. According to [Wikipedia](https://en.wikipedia.org/wiki/Stock_market_prediction), **stock market prediction** is defined as:

> [...] the act of trying to determine the future value of a company stock or other financial instrument traded on an exchange. The successful prediction of a stock's future price could yield significant profit.

Needless to say, the potential returns for a successful approach in this direction is of great interest for financial analysts and traders. However, there is an on-going mathematical and philosophical debate on [whether it is even possible to predict anything about the future evolution of stocks](https://en.m.wikipedia.org/wiki/Random_walk_hypothesis), beyond pure guessing:

> The efficient-market hypothesis suggests that stock prices reflect all currently available information and any price changes that are not based on newly revealed information thus are inherently unpredictable. Others disagree and those with this viewpoint possess myriad methods and technologies which purportedly allow them to gain future price information.

While the truth might be somewhere in the middle, we'd like to do our best to analyse and predict future evolutions of the stock market.

## References

In preparing our submission, we've referred to the following materials:

- Wikipedia articles on [stock market prediction](https://en.wikipedia.org/wiki/Stock_market_prediction), the [efficient-market hyptothesis](https://en.wikipedia.org/wiki/Efficient-market_hypothesis)

- TensorFlow's [Time series forecasting tutorial](https://www.tensorflow.org/tutorials/structured_data/time_series)

- Neptune's [Predicting Stock Prices Using Machine Learning](https://neptune.ai/blog/predicting-stock-prices-using-machine-learning)

- Abagen's [Data normalization options](https://abagen.readthedocs.io/en/stable/user_guide/normalization.html)

- [Stock Closing Price Prediction using Machine Learning Techniques](https://www.sciencedirect.com/science/article/pii/S1877050920307924) by M. Vijha, D. Chandolab, V. A. Tikkiwalb and A. Kumarc

- [Stock Price Forecasting by a Deep Convolutional Generative Adversarial Network](https://www.frontiersin.org/articles/10.3389/frai.2022.837596/full) by A. Staffini

- [Impact of Data Normalization on Stock Index Forecasting](https://www.researchgate.net/publication/291962265_Impact_of_Data_Normalization_on_Stock_Index_Forecasting) by S. C. Nayak

- Pandas's [Market Calendars](https://pandas-market-calendars.readthedocs.io/en/latest/usage.html), Kiplinger's [Stock Exchange Holidays](https://www.kiplinger.com/investing/603728/stock-market-holidays-in-2022) and TDS's [Holidays Calendars Article](https://towardsdatascience.com/holiday-calendars-with-pandas-9c01f1ee5fee)

- Medium's ([1](https://medium.datadriveninvestor.com/step-by-step-time-series-analysis-d2f117554d7e)), TDS's ([1](https://towardsdatascience.com/how-to-remove-non-stationarity-in-time-series-forecasting-563c05c4bfc7),[2](https://towardsdatascience.com/stationarity-assumption-in-time-series-data-67ec93d0f2f)) and MLM's ([1](https://machinelearningmastery.com/remove-trends-seasonality-difference-transform-python/#:~:text=Time%20series%20are%20stationary%20if,the%20variance%20of%20the%20observations.),[2](https://machinelearningmastery.com/difference-time-series-dataset-python/)) Stationarity Time Series Articles

## Further improvements

The following is a non-exhaustive list of ideas on how this project could be expanded and improved.

- Define and use _all_ of the extra predictive variables which were introduced in [[1](https://www.sciencedirect.com/science/article/pii/S1877050920307924)] and [[2](https://www.frontiersin.org/articles/10.3389/frai.2022.837596/full)].

- Detect price [support and resistance](https://www.investopedia.com/trading/support-and-resistance-basics/) levels using code, based on the approach described [here](https://towardsdatascience.com/detection-of-price-support-and-resistance-levels-in-python-baedc44c34c9).

- Add a new price rolling mean, but weighted by **the volume of transactions** for each previous day.

- Define new input variables using [exponentially weighted rolling means](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html) (natively supported by `pandas`)

- Define a new variable using [Spencer's 15-point weighted moving average](https://mathworld.wolfram.com/Spencers15-PointMovingAverage.html), which is sometimes used by actuaries.

- Implement [RMSE](https://en.wikipedia.org/wiki/Root-mean-square_deviation) as an error function.

- Fine-tune the [XGBoost](https://en.wikipedia.org/wiki/XGBoost) model to run faster and produce better results. The `XGBRegressor` class has quite a few tunable hyperparameters, which are presented in [the XGBoost documentation](https://xgboost.readthedocs.io/en/stable/index.html).

- Use deep learning algorithms for stock closing price prediction:
  - The TensorFlow tutorial on [Time series forecasting](https://www.tensorflow.org/tutorials/structured_data/time_series) provides code for the following models:
    - Basic neural network
    - Convolutional neural network
    - [Long short-term memory](https://www.simplilearn.com/tutorials/machine-learning-tutorial/stock-price-prediction-using-machine-learning)
  - The GAN-based method from [this research paper](https://www.frontiersin.org/articles/10.3389/frai.2022.837596/full)

## Data loading

In this section we obtain a path to the given data file and load it into memory.

In [ ]:
from pathlib import Path

# filename = 'DataSet_Target Portfolio.xlsx'
filename = 'DataSet_Target Portfolio - exported on 19th of April.xlsx'

# Check if we're running on Colab or not
running_on_google_colab = 'google.colab' in str(get_ipython())
if running_on_google_colab:
    from google.colab import files

    dataset_file = Path(filename)
    if not dataset_file.is_file():
        files.upload()
else:
    dataset_file = Path('/mnt/d/Libraries/Downloads/') / Path(filename)

assert dataset_file.is_file(), 'Could not open dataset file!'

In [ ]:
import pandas as pd

In [ ]:
column_names = ('date', 'open', 'high', 'low', 'close', 'adj_close', 'volume')
index_column_name = column_names[0]
sheet_names = ('GS', 'C', 'WFC', 'BAC', 'JPM')

datasets = {}
for sheet_name in sheet_names:
    dataset = pd.read_excel(dataset_file,
                            names=column_names,
                            index_col=index_column_name,
                            sheet_name=sheet_name)
    datasets[sheet_name] = dataset

print('Successfully read stock data for', len(datasets), 'companies:')
print(' ', ', '.join(datasets.keys()))

Since making all the plots for the companies in the input data in the same notebook would be tiresome, we're going to pick one of them and store it in the `dataset` variable, which we will reuse throughout the notebook:

In [ ]:
dataset = datasets['GS']
dataset

## Exploratory data analysis

### Summary statisics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['figure.dpi'] = 120

sns.set_style("whitegrid")

In [ ]:
dataset.describe().transpose()

In [ ]:
dataset.shape

In [ ]:
dataset.open.hist()
plt.title("Open values")
plt.show()

Getting a sense of the series:

In [ ]:
subset = dataset.iloc[0:]

fig, ax = plt.subplots()

ax.plot(subset.index, subset.open, label = 'open')
ax.plot(subset.index, subset.high, label = 'high')
ax.plot(subset.index, subset.low, label = 'low')

plt.legend()
plt.show()

In [ ]:
plt.plot(dataset.index, np.abs(dataset.close - dataset.adj_close))
plt.title("The difference between Close and Adjusted Close")
plt.show()

In [ ]:
plt.suptitle('Volume of transactions over time')

plt.plot(dataset.index, dataset.volume)

plt.ylabel('Volume (units)')
plt.xlabel('Date')

plt.show()

### Missing data

Look for missing values in the data:

In [ ]:
dataset.isna().sum()

Generate all the bussiness days between the first and last entry to check if there are missing days:

In [ ]:
start_date = dataset.head(1).index[0]
end_date = dataset.tail(1).index[0]

dates = pd.bdate_range(start=start_date, end=end_date)

print("The difference in the number of days: " + str(abs(dates.shape[0] - dataset.shape[0])))

We considered only the weekdays for the range. Let's get all the holidays out of the working range:

In [ ]:
!pip install --upgrade pandas-market-calendars

In [ ]:
import pandas_market_calendars as mcal

nyse = mcal.get_calendar('NYSE')
dates = nyse.valid_days(start_date=start_date, end_date=end_date)

holidays_missmatch = False
for date in range(dates.shape[0]):
  if str(dataset.index[date])[:10] != str(dates[date])[:10]:
    holidays_missmatch = True
    break

if holidays_missmatch:
  print("There is a mismatch between the dataset and holidays dataset!")
else:
  print("The dataset matches the holidays dataset!")


We have used the NYSE holidays for the matching. However, if we want to check for a different series of holidays, we can manually insert them as so:

In [ ]:
from pandas.tseries.holiday import *
from pandas.tseries.offsets import CustomBusinessDay

class HolidaysCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Year', month = 1, day = 1, observance = sunday_to_monday),
        Holiday('Groundhog Day', month = 1, day = 6, observance = sunday_to_monday),
        Holiday('St. Patricks Day', month = 3, day = 17, observance = sunday_to_monday),
        Holiday('April Fools Day', month = 4, day = 1),
        Holiday('Good Friday', month = 1, day = 1, offset = [Easter(), Day(-2)]),
        Holiday('Labor Day', month = 5, day = 1, observance = sunday_to_monday),
        Holiday('Canada Day', month = 7, day = 1, observance = sunday_to_monday),
        Holiday('July 4th', month = 7, day = 4, observance = nearest_workday),
        Holiday('All Saints Day', month = 11, day = 1, observance = sunday_to_monday),
        Holiday('Christmas', month = 12, day = 25, observance = nearest_workday)
    ]

_holidays = CustomBusinessDay(calendar = HolidaysCalendar())
_dates = pd.bdate_range(start = start_date, end = end_date, freq = _holidays)
_dates = pd.DataFrame(_dates, columns = ['date'])

df = dataset.index - _dates['date']
# print("Are there any mismatches? " + str(df.any()))

### Checking for outliers

We will check for outliers using the [ThymeBoost](https://towardsdatascience.com/time-series-outlier-detection-with-thymeboost-ec2046e17458) library. ThymeBoost is a toolkit for time series decomposition using gradient boosting tehniques, but we'll use it specifically for extracting valuable information about stationarity, seasonality, etc.

We first have to install this library, if it's not already present:

In [ ]:
!pip install --upgrade --quiet ThymeBoost

Now we can create a new boosted model, and use it to detect outliers in the stock's closing prices:

In [ ]:
from ThymeBoost import ThymeBoost as tb

boosted_model = tb.ThymeBoost()
output = boosted_model.detect_outliers(dataset.close.values,
                                       trend_estimator='linear',
                                       seasonal_estimator='fourier',
                                       seasonal_period=25,
                                       global_cost='maicc',
                                       fit_type='global')

In [ ]:
boosted_model.plot_results(output)
boosted_model.plot_components(output)

In [ ]:
dy = output.trend[1] - output.trend[0]
dx = 1 - 0
slope = dy/dx

print('slope:', slope)

The plots created by ThymeBoost are fairly informative:

- The stock market has an overall ascending trend, with a slope of about $0.07$.

- Besides the smaller usual variations of the closing market price, we see that the biggest outliers found by ThymeBoost are in in the first quarters of 2020 and 2022. The first one has been caused by the [CoVID pandemic](https://en.wikipedia.org/wiki/List_of_stock_market_crashes_and_bear_markets) (2020), while the second one is an inflationary wave (caused by the post-pandemic economic recovery effort), ended by the Russian invasion of Ukraine.

- There doesn't seem to be any discernible seasonality in the data; seasonal effects look like random noise.

We can also try to identify different local patterns in the charts (e.g. the head and shoulders pattern) and correlate them with real-life economic events.

### Trying to detect seasonality in the data

Predicting the stock market is very different from predicting the weather; there is no discernible pattern in the data, since the growth of the economy doesn't really depend on the season or the day of the week. That being said, we can use the [Fourier transform](https://en.wikipedia.org/wiki/Fourier_transform) to convince ourselves that there really isn't any sort of recurring behavior, no matter the level of the scale we're looking at:

In [ ]:
closing_price_frequency_spectrum = np.fft.rfft(dataset.close)

closing_price_spectrum_abs = np.abs(closing_price_frequency_spectrum)
closing_price_spectrum_abs /= np.max(closing_price_spectrum_abs)

plt.plot(closing_price_spectrum_abs)

plt.ylabel('Closing price value (normalized)')

plt.xscale('log')
plt.xlabel('Frequency (log scale)')

plt.show()

In [ ]:
closing_price_deltas = np.abs(np.diff(dataset.close))
closing_price_deltas_frequency_spectrum = np.fft.rfft(closing_price_deltas)

closing_price_deltas_abs = np.abs(closing_price_deltas_frequency_spectrum)
closing_price_deltas_abs /= np.max(closing_price_deltas_abs)

plt.plot(closing_price_deltas_abs)

plt.ylabel('Closing price delta (normalized)')

plt.xscale('log')
plt.xlabel('Frequency (log scale)')

plt.show()

In [ ]:
volume_frequency_spectrum = np.fft.rfft(dataset.volume)

volume_spectrum_abs = np.abs(volume_frequency_spectrum)
volume_spectrum_abs /= np.max(volume_spectrum_abs)

plt.plot(volume_spectrum_abs)

plt.ylabel('Volume (normalized)')

plt.xscale('log')
plt.xlabel('Frequency (log scale)')

plt.show()

### The stationarity of the data

[Stationary data](https://towardsdatascience.com/stationarity-assumption-in-time-series-data-67ec93d0f2f) refers to the time series data that mean and variance do not vary across time. The data is considered non-stationary if there is a strong trend or seasonality observed from the data.

Using non-stationary time series data in financial models produces unreliable and spurious results and leads to poor understanding and [forecasting](https://www.researchgate.net/post/Is_the_stock_return_series_ALWAYS_stationary). The solution to the problem is to transform the time series data so that it becomes stationary.

We will use the Augmented Dickey-Fuller test to check for the stationarity of our data. We want a p-value lower than $0.05$.

In [ ]:
from statsmodels.tsa.stattools import adfuller

test_results = adfuller(dataset["close"])

print(f"ADF test statistic: {test_results[0]}")
print(f"p-value: {test_results[1]}")
print("Critical thresholds:")

for key, value in test_results[4].items():
    print(f"\t{key}: {value}")

For the ADF statistical test we have the following:
- *The null hypothesis*: the distribution is non-stationary, time-dependent (it has a unit root).
- *The alternative hypothesis*: the distribution is stationary, not time-dependent (can’t be represented by a unit root).

The p-value determines the result of the test. If it is smaller than a critical threshold of 0.05 or 0.01, we reject the null hypothesis and conclude that the series is stationary. Otherwise, we fail to reject the null and conclude the series is non-stationary.

As expected, we have a high p-value. Let's try to check the differencing of the time series:

In [ ]:
data = dataset["close"].copy()

plt.plot(data)
plt.title("The time series")
plt.show()

data_difference = data.diff()
plt.plot(data_difference)
plt.title("The first difference time series")
plt.show()

data_difference_second = data_difference.diff()
plt.plot(data_difference_second)
plt.title("The second difference time series")
plt.show()

It looks like a single differencing process is enough. Let's repeat the ADF test for the once differenced time series:

In [ ]:
# Get rid of the first value (NaN)
data_difference[0] = data_difference[1]

test_results = adfuller(data_difference)

print(f"ADF test statistic: {test_results[0]}")
print(f"p-value: {test_results[1]}")
print("Critical thresholds:")

for key, value in test_results[4].items():
    print(f"\t{key}: {value}")

Our p-value is almost 0. This means we can easily reject the null hypothesis and consider the distribution as stationary.

The difference will be done manually, so we can reverse the changes at the end.

In [ ]:
def difference(dataset, interval = 1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	diff.insert(0, np.nan)
	return diff
 
def inverse_difference(last_ob, value):
	return value + last_ob
 
data = data.values
plt.plot(data)
plt.title("The time series")
plt.show()

diff = difference(data)
plt.plot(diff)
plt.title("The difference time series")
plt.show()

inverted = [inverse_difference(data[i], diff[i]) for i in range(len(diff))]
inverted.insert(0, data[0])
plt.plot(inverted)
plt.title("The inverted difference time series")
plt.show()

In [ ]:
_ = diff.pop(0)

Let's check the *ThymeBoost* statistics on the differenced time series:

In [ ]:
boosted_model = tb.ThymeBoost()
output = boosted_model.detect_outliers(diff,
                                       trend_estimator = 'linear',
                                       seasonal_estimator = 'fourier',
                                       seasonal_period = 25,
                                       global_cost = 'maicc',
                                       fit_type = 'global')
boosted_model.plot_results(output)
boosted_model.plot_components(output)

## Data range

It looks like we need to normalize the data, which we'll do anyway before putting it into the models.

In [ ]:
dataset.volume.hist()
plt.title("Volume values")
plt.plot()

## Data augmentation


### Additional variables

Besides the raw timeseries data, traders who perform technical analysis on stock prices also extract from the data some additional aggregate statistics, (potentially weighted) moving averages and other derived indicators. We'll do so as well, with the intention of providing additional information to the  statistical models.

The additional variables we define in these two papers [[1](https://www.sciencedirect.com/science/article/pii/S1877050920307924), [2](https://www.frontiersin.org/articles/10.3389/frai.2022.837596/full)] . We did not do ablation tests to verify if each of these additional variables helps improve the models' predicitive power, instead expecting the models to assign lesser weights to less relevant input columns.

In [ ]:
dataset['high_low_difference'] = dataset['high'] - dataset['low']
dataset['close_open_difference'] = dataset['close'] - dataset['open']
dataset['seven_days_moving_average'] = dataset['close'].rolling(7, closed='both').mean().fillna(dataset['close'].iloc[0])
dataset['fourteen_days_moving_average'] = dataset['close'].rolling(14, closed='both').mean().fillna(dataset['close'].iloc[0])
dataset['twenty_one_days_moving_average'] = dataset['close'].rolling(21, closed='both').mean().fillna(dataset['close'].iloc[0])
dataset['seven_days_stddev_moving_average'] = dataset['close'].rolling(7, closed='both').std().fillna(1)

dataset['close_diff'] = difference(dataset['close'])

Let's take a quick look at how these new variables look like:

In [ ]:
subset = dataset[-500:]

plt.plot(subset.index, subset['close'], label='closing price')
plt.plot(subset.index, subset['seven_days_moving_average'], label='7-day moving average')
plt.plot(subset.index, subset['fourteen_days_moving_average'], label='14-day moving average')
plt.plot(subset.index, subset['twenty_one_days_moving_average'], label='21-day moving average')

plt.legend()

plt.show()

Let's define some constants for our variables available for making our prediction:

In [ ]:
AVAILABLE_VARIABLES = [
    'open', 'close', 'adj_close', 'high', 'low', 'volume',
    'high_low_difference', 'close_open_difference',
    'seven_days_moving_average',
    'fourteen_days_moving_average',
    'twenty_one_days_moving_average',
    'seven_days_stddev_moving_average',
    'close_diff',
]

### Differencing

In [ ]:
def compute_differences(arr):
    diffs = np.diff(arr)
    return np.insert(diffs, 0, 0)

def reverse_differences(diffs, offset):
    diffs = np.delete(diffs, 0)
    return np.cumsum(np.insert(diffs, 0, offset))

In [ ]:
differences = compute_differences(dataset.close)

In [ ]:
figure, (ax1, ax2) = plt.subplots(2, 1)
figure.suptitle('Histogram of closing prices vs. closing price deltas')

sns.histplot(dataset.close, stat='probability', ax=ax1)
sns.histplot(differences, kde=True, stat='probability', ax=ax2)

plt.show()

In [ ]:
reverted = reverse_differences(differences, dataset.close[0])

In [ ]:
plt.plot(np.arange(len(dataset)), dataset.close)
plt.plot(np.arange(len(dataset)), reverted)
plt.show()

In [ ]:
np.abs(reverted - dataset.close).max()

In [ ]:
#dataset.close = compute_differences(dataset.close)

## Building the validation set

Before training any models, we'll first split our given data into two parts: the first 2100 samples will be used for training the model to predict the future, and the last ~400 samples will be novel, unseen data on which we will evaluate the models' performance.

In [ ]:
training_set = dataset.iloc[:2100].copy()
validation_set = dataset.iloc[2100:].copy()

We can see that the split leads to the validation dataset starting in August 2020, after CoVID hit and the financial markets started adapting to the new reality:

In [ ]:
validation_set_first_timestamp = validation_set.index[0]

print(validation_set_first_timestamp.strftime('%Y-%m-%d'))

## Data normalization

To make model training more efficient and to improve its predictive performance, data is usually **normalized** before being fed into a machine learning algorithm. This is done to ensure that the (absolute) variations in the data points are at about the same scale, improving the numerical stability of the machine learning methods we'll use.

All of these normalization functions are reversible.

### Determining which kind of normalization to use

**Min-Max Normalization:** Normalizes the data, converting the closed inverval $[\min, \max]$ into $[0, 1]$. The main problem with this method is that we could have values outside of our interval in the future. However, given a dataset that spans a long time interval, the assumption that new values do not lie outside our interval is usually accepted.

$$ \hat{x} = \frac{x - min}{max - min} $$

In [ ]:
def normalize_min_max(training_set, validation_set):

  min = training_set.min()
  max = training_set.max()

  training_set = (training_set - min) / (max - min)
  validation_set = (validation_set - min) / (max - min)

  return training_set, validation_set

**Z-Score Normalization:** Normalizes the data, converting the mean to 0 and the standard deviation to 1. Best used when the data is normally distributed and the time series is stationary.

$$ \hat{x} = \frac{x - \mu}{\sigma} $$

In [ ]:
def normalize_mean_std(training_set, validation_set):

  mean = training_set.mean()
  std = training_set.std()

  training_set = (training_set - mean) / std
  validation_set = (validation_set - mean) / std

  return training_set, validation_set

**Median Value Normalization:** Normalizes the data by dividing each sample by the median. The median is not influenced by the magnitude of extreme deviations, so it can handle outliers with ease.

$$ \hat{x} = \frac{x}{\text{median}(x)} $$

In [ ]:
def normalize_mean(training_set, validation_set):

  median = training_set.median()

  training_set = training_set / median
  validation_set = validation_set / median

  return training_set, validation_set

**Sigmoid Normalization:** Normalizes the data by replacing each sample $s$ with $(1-e^s)^{-1}$. The working interval becomes $[0,1]$ and this normalization method is great for when we do not know the underlying distribution of our data. We used the a standard sigmoid here, but it is worth mentioning that there are other similar ways to do this task (i.e. scaled sigmoid, robust sigmoid, etc.).

$$ \hat{x} = \frac{1}{1 - e^{x}} $$

In [ ]:
def normalize_sigmoid(training_set, validation_set):

  training_set = 1 / (1 - np.exp(1) ** training_set)
  validation_set = 1 / (1 - np.exp(1) ** validation_set) 

  return training_set, validation_set

There are multiple other ways to normalize the data (e.g. arctangent estimators, median and median absolute deviation, etc.). However, for the given task, it seems that the optimal choice is the **Min-Max Normalization**, so this will be the method we will use in building the models. The implementation used will be the one provided by *sklearn*, as it already implements all the needed functionality.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

values = pd.DataFrame(training_set['close'].values)
scaler = MinMaxScaler(feature_range = (0, 1))
scaler = scaler.fit(values)

print('Min: %f\nMax: %f' % (scaler.data_min_, scaler.data_max_))

In [ ]:
training_set

Normalize the values and check the result:

In [ ]:
normalized_values = scaler.transform(values)

normalized_values

Restore the normalized values and check the result:

In [ ]:
restored_values = scaler.inverse_transform(normalized_values)

restored_values

### Normalizing the dataset

In [ ]:
scalers = {}

for variable in AVAILABLE_VARIABLES:
    scalers[variable] = MinMaxScaler(feature_range=(0, 1))
    scalers[variable] = scalers[variable].fit(dataset[variable].to_numpy().reshape(-1, 1))

    training_set[variable] = scalers[variable].transform(pd.DataFrame(training_set[variable].values))
    validation_set[variable] = scalers[variable].transform(pd.DataFrame(validation_set[variable].values))

In [ ]:
training_set

## Detecting outliers

Drop the NaN's created by the differencing:

In [ ]:
training_set = training_set.dropna()
validation_set = validation_set.dropna()

In [ ]:
# Restore the data
# training_set['close'] = scaler.inverse_transform(pd.DataFrame(training_set['close'].values))
# validation_set['close'] = scaler.inverse_transform(pd.DataFrame(validation_set['close'].values))

In [ ]:
boosted_model = tb.ThymeBoost()
output = boosted_model.detect_outliers(training_set['close'].values,
                                       trend_estimator = 'linear',
                                       seasonal_estimator = 'fourier',
                                       seasonal_period = 25,
                                       global_cost = 'maicc',
                                       fit_type = 'global')
boosted_model.plot_results(output)
boosted_model.plot_components(output)

We can now create a violin plot of the values in the training set and see that they're almost normally distributed around 0:

In [ ]:
df = training_set.melt(var_name='Column', value_name='Normalized')
ax = sns.violinplot(x='Column', y='Normalized', data=df)
ax.set_xticklabels(training_set.keys(), rotation=90)
plt.show()

## Model construction and evaluation

Now that we have a much better understanding of the data we're working with and have set up the scene for further experimentation on it, in this section we will attempt to construct a few **models** to help us predict the future based on patterns we've seen in the past.

The libraries we are going to use will be `sklearn` (for simpler, linear regression-based models, as well as random forests) and `xgboost` (for gradient-boosted ensembles of trees).

In [ ]:
import sklearn
import xgboost
import tensorflow as tf

We'll start by storing the name of the variable (column) we want to predict in a Python constant. We're interested in forecasting each future day's closing stock price, since that's a relevant indicator for understanding the evolution of a company's performance over time.

In [ ]:
TARGET_VARIABLE = 'close'

### Error function

In order to evaluate the performance of our models, we need to define an **error function**, which will help us compute a numerical value, describing how far off our predictions are from reality.

According to Wikipedia, a [forecasting error](https://en.wikipedia.org/wiki/Forecast_error) is

> the difference between the actual or real and the predicted or forecast value of a time series [..]

We're not going to use the differences between the real and predicted values directly, rather we'll put them through some formulas (described below).

#### Mean absolute error

One of the simplest error functions to understand is [the mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error). We take the differences between every forecasted value and the real value, compute the absolute value of each difference and sum these up.

$$\mathrm{MAE} = \frac{1}{n} \sum_{i = 1}^{n} \lvert y_i - \hat{y}_i \rvert$$ 

In [ ]:
def mean_absolute_error(expected, predicted):
    differences = expected - predicted
    absolute_differences = np.absolute(differences)
    return absolute_differences.mean()

#### Mean squared error

Another straightforward error function, often used in regression problems, is [the mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error). Just like before, we compute the elementwise differences between the target values and the forecasts, then square them and sum them up.

$$\mathrm{MSE} = \frac{1}{n} \sum_{i = 1}^{n} (y_i - \hat{y}_i)^2$$

In [ ]:
def mean_squared_error(expected, predicted):
    differences = expected - predicted
    squared_differences = differences ** 2
    return squared_differences.mean()

#### Mean absolute percentage error

An error function which is quite popular in forecasting is [the mean absolute percentage error](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error). It's not very difficult to define and has an intuitive interpretation (it's just like the MAE, but offers a percentual/relative error).

$$\mathrm{MAPE} = 100\% \cdot \frac{1}{n} \sum_{i = 1}^{n} \left\lvert\frac{y_i - \hat{y}_i}{y_i}\right\rvert$$

Due to it's formulation, MAPE cannot be used in situations where the predicted value is zero. Fortunately, none of our stock prices ever go down to values close to nil.

In [ ]:
def mean_absolute_percentage_error(expected, predicted):
    differences = expected - predicted
    relative_absolute_differences = np.absolute(differences / expected)
    return relative_absolute_differences.mean() * 100

### Mean absolute scaled error

[The mean absolute scaled error](https://en.wikipedia.org/wiki/Mean_absolute_scaled_error) is similar to the mean absolute error, but addresses some of it's limitations:

- it's independent of the scale of the data
- it works correctly even when the difference between the predicted value and the real value tends to zero (unlike the MAPE)
- it's easier to interpret; if the error is greater than one, it means that the prediction is worse than a "baseline" prediction of just assuming that the stock price stayed constant compared to the previous day

$$\mathrm{MASE} = \frac{\frac{1}{n} \sum_{i = 1}^{n} \lvert y_i - \hat{y}_i \rvert}{\frac{1}{n - 1} \sum_{j = 2}^{n} \lvert y_j - y_{j - 1} \rvert}$$

In [ ]:
def mean_absolute_scaled_error(expected, predicted):
    differences = expected - predicted
    absolute_differences = np.absolute(differences)
    baseline_differences = np.diff(expected)
    return absolute_differences.mean() / baseline_differences.mean()

#### Putting them all together

Since we don't now from the start which error functions will be better predictors of actual model performance, we're also going to define a helper function which will compute all of the pre-defined metrics for a model's output. We'll use these later when comparing the accuracy of the models.

In [ ]:
ERRORS = ['MAE', 'MSE', 'MAPE', 'MASE']

def compute_errors(expected, predicted):
    return {
        'MAE': mean_absolute_error(expected, predicted),
        'MSE': mean_squared_error(expected, predicted),
        'MAPE': mean_absolute_percentage_error(expected, predicted),
        'MASE': mean_absolute_scaled_error(expected, predicted),
    }

### Data windowing

Considering and the nature of this challenge, and to reflect a realistic use case for the constructed models, we're going to use the following data windowing pattern during model validation:

- The models are trained on all available data, up to the last day before the forecasts are made.
- The models have to predict the closing price of the respective stocks for the next 5 days (ignoring market holidays).
- Afterwards, they're given access to the real data for the 5 predicted days, are refitted and the process is repeated.

In [ ]:
WINDOW_SIZE = 5

### Model API

To make it simpler to construct, train, evaluate and use the models we're going to design, we shall first define an interface for all models to implement:

In [ ]:
from abc import ABC, abstractmethod

class StockPriceForecastingModel(ABC):
    @abstractmethod
    def train(self, data, delta):
        '''Trains the model on all of the available data up to
        the current validation timestamp.
        
        - `data` is a pandas `DataFrame` which contains (at the least) the columns
        `open`, `close`, `low`, `high` and `volume`.
        - `delta` is a pandas `DataFrame` containing only the most recently added data,
        compared to the rows of data from the last call. It is `None` on the first call to `train`.
        '''

    @abstractmethod
    def predict(self, timestamps):
        '''Requests the model to predict the closing price of the input stock
        for five consecutive days.
        
        The dates are passed in as an array of pandas `Timestamp`s. The model
        must return an array of predicted stock prices (real numbers).
        '''

We're also going to extract the common training and evaluation code into a reusable function:

In [ ]:
import math
from tqdm import trange

def train_and_evaluate_model(model):
    current_training_set = training_set.copy()

    # Initial (cold) training of the model on the training set.
    model.train(current_training_set, None)

    # Record the predictions the model makes on each window, for plotting.
    predictions = []

    # Record the history of error values as we progress through the validation set.
    errors = []

    window_num = 1
    window_start = 0
    window_end = window_start + WINDOW_SIZE

    num_windows = math.ceil(len(validation_set) / WINDOW_SIZE)

    for window_index in trange(num_windows):
        window_start = window_index * WINDOW_SIZE
        window_end = (window_index + 1) * WINDOW_SIZE

        # Extract the window of data from the validation set
        window = validation_set.iloc[window_start:window_end]

        # Make some forecasts
        predicted = model.predict(window.index)

        # Check that the model respect's the `StockPriceForecastingModel` interface.
        assert isinstance(predicted, pd.Series), \
            '`predict` must return a pandas `Series`'

        assert (window.index == predicted.index).all(), \
            'returned series must have same index as given as input'

        predictions.append(predicted)

        # Evaluate the results
        expected = window.close
        window_errors = compute_errors(expected, predicted)
        
        #print(f'Window #{window_num}:', window_errors)

        errors.append(window_errors)

        # Now expand the available dataset
        current_training_set = pd.concat((current_training_set, window))

        # And allow the model to improve it's understanding
        model.train(current_training_set, window)

    # Take the list of dictionaries describing the prediction errors on each window
    # and turn it into a DataFrame
    errors = pd.DataFrame(errors)

    # Return the arithmetic mean of the prediction error across all windows.
    # This is OK since our errors were already arithemtic means of some other values.
    mean_error = errors.mean()

    return predictions, errors, mean_error

In [ ]:
def plot_predictions(predictions):
    complete_forecast = pd.concat(predictions).to_numpy().reshape(1, -1)
    complete_forecast = scalers[TARGET_VARIABLE].inverse_transform(complete_forecast)

    #offset = training_set.close.iloc[0]
    #complete_forecast = reverse_differences(complete_forecast, offset)

    expected_values = validation_set[TARGET_VARIABLE].to_numpy().reshape(1, -1)
    expected_values = scalers[TARGET_VARIABLE].inverse_transform(expected_values)
    #expected_values = reverse_differences(expected_values, offset)

    x = validation_set.index
    plt.plot(x, expected_values.reshape(-1))
    plt.plot(x, complete_forecast.reshape(-1))

In [ ]:
def plot_errors(errors):
    """Plots the values attained by the error functions as the model
    progressed through the validation dataset. 
    
    `errors` must be a pandas `DataFrame` where the columns are the names of
    the error functions, as defined in the `ERRORS` constant, and the rows are
    the values of the corresponding error functions.
    """
    fig, ax = plt.subplots(nrows=len(ERRORS))

    for index, error in enumerate(ERRORS):
        ax[index].plot(errors[error], label=error)

    return fig, ax

### Baseline model - constant prediction



For a baseline, we'll simply take the values from the last day we have in the training set and predict that the stock market will stay constant after that ([the naïve approach to forecasting](https://en.wikipedia.org/wiki/Forecasting#Na.C3.AFve_approach)):

In [ ]:
class ConstantStockPriceModel(StockPriceForecastingModel):
    def train(self, data, delta):
        self.last_closing_price = data[TARGET_VARIABLE].iloc[-1]

    def predict(self, timestamps):
        forecasted_closing_prices = np.tile(self.last_closing_price, timestamps.shape)
        return pd.Series(index=timestamps, data=forecasted_closing_prices)

In [ ]:
model = ConstantStockPriceModel()

predictions, errors, mean_error = train_and_evaluate_model(model)

In [ ]:
plot_predictions(predictions)
plt.show()

In [ ]:
fig, _ = plot_errors(errors)
fig.suptitle('Baseline/constant model errors')
plt.show()

In [ ]:
mean_error

### Common code for sklearn-based models

In [ ]:
class SklearnModel(StockPriceForecastingModel):
    def train(self, data, delta):
        # Determine how many full windows of data we can generate from the given training data.
        num_windows = len(data) // WINDOW_SIZE - 1

        # Since the length of the training data might not be evenly divisible by the length of a window,
        # we'll have to start from an offset.
        offset = len(data) % WINDOW_SIZE

        # Extract a subset of the data, to be used for creating windows of input/target variables.
        available_training_data = data[AVAILABLE_VARIABLES].iloc[offset:offset + num_windows * WINDOW_SIZE].to_numpy()
        available_target_data = data[TARGET_VARIABLE].iloc[offset + WINDOW_SIZE:offset + WINDOW_SIZE + num_windows * WINDOW_SIZE].to_numpy()

        row_size = WINDOW_SIZE * len(AVAILABLE_VARIABLES)

        X = available_training_data.reshape(num_windows, row_size)
        y = available_target_data.reshape(num_windows, WINDOW_SIZE)

        # Fit the model on the data.        
        self.model.fit(X, y)

        # The last window of available data will be used for prediction.
        self.last_window = data[AVAILABLE_VARIABLES].iloc[-WINDOW_SIZE:].to_numpy().reshape(1, row_size)

    def predict(self, timestamps):
        forecasted_closing_prices = self.model.predict(self.last_window)
        forecasted_closing_prices = forecasted_closing_prices[0]

        # Since the validation dataset might not be broken up evenly into windows,
        # we could be asked to forecast the closing stock price for a period of time
        # shorter than 5 days.
        forecasted_closing_prices = forecasted_closing_prices[:len(timestamps)]

        return pd.Series(index=timestamps, data=forecasted_closing_prices)

### Linear regression

Another very simple kind of model is linear regression. It will take as input only the last five days of data to predict the next five days.

In [ ]:
from sklearn.linear_model import LinearRegression

class LinearRegressionModel(SklearnModel):
    def __init__(self):
        self.model = LinearRegression()

In [ ]:
model = LinearRegressionModel()

predictions, errors, mean_error = train_and_evaluate_model(model)

In [ ]:
plot_predictions(predictions)
plt.show()

In [ ]:
fig, _ = plot_errors(errors)
fig.suptitle('Linear regression model errors')
plt.show()

In [ ]:
mean_error

### Random forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

class RandomForestModel(SklearnModel):
    def __init__(self):
        self.model = RandomForestRegressor(max_samples=0.5)

In [ ]:
model = RandomForestModel()

predictions, errors, mean_error = train_and_evaluate_model(model)

In [ ]:
plot_predictions(predictions)
plt.show()

In [ ]:
fig, _ = plot_errors(errors)
fig.suptitle('Random forest model errors')
plt.show()

In [ ]:
mean_error

### Gradient-boosted trees

In [ ]:
!pip install --upgrade xgboost>=1.6

In [ ]:
from xgboost import XGBRegressor

class XGBRegressorModel(SklearnModel):
    def __init__(self):
        self.model = XGBRegressor(objective='reg:squarederror', subsample=0.25)

In [ ]:
model = XGBRegressorModel()

predictions, errors, mean_error = train_and_evaluate_model(model)

In [ ]:
plot_predictions(predictions)
plt.show()

In [ ]:
fig, _ = plot_errors(errors)
fig.suptitle('XGB regressor model errors')
plt.show()

In [ ]:
mean_error

# Output file

In [ ]:
sheet_names = ('GS', 'C', 'WFC', 'BAC', 'JPM')
dates = ('21-04-22', '22-04-22', '25-04-22', '26-04-22', '27-04-22')